In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [ ]:
#ls

In [4]:
data = spark.read.csv("smsspamcollection/SMSSpamCollection", inferSchema=True, sep="\t")

In [9]:
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

In [10]:
data.show(5)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
+-----+--------------------+
only showing top 5 rows



In [11]:
from pyspark.sql.functions import length

In [12]:
data = data.withColumn('length', length(data['text']))

In [13]:
data.show(5)

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
+-----+--------------------+------+
only showing top 5 rows



In [14]:
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



### Feature Transformations

In [15]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [16]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stopremove = StopWordsRemover(inputCol='token_text', outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_num = StringIndexer(inputCol='class', outputCol='label')

In [17]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [18]:
clean_up = VectorAssembler(inputCols=['tf_idf', 'length'],
                          outputCol='features')

### The Model

In [19]:
from pyspark.ml.classification import NaiveBayes

In [20]:
nb = NaiveBayes()

### Pipeline

In [23]:
from pyspark.ml import Pipeline

In [24]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,
                                 tokenizer,
                                 stopremove,
                                 count_vec,
                                 idf,
                                 clean_up])

In [25]:
cleaner = data_prep_pipe.fit(data)

In [26]:
clean_data = cleaner.transform(data)

### Training and Evaluation

In [27]:
clean_data = clean_data.select(['label', 'features'])

In [29]:
clean_data.show(10)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
+-----+--------------------+
only showing top 10 rows



In [30]:
(training, testing) = clean_data.randomSplit([0.7, 0.3])

In [31]:
spam_predictor = nb.fit(training)

In [32]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [33]:
test_results = spam_predictor.transform(testing)

In [34]:
test_results.show(10)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,13,...|[-613.51977741565...|[0.99999999999999...|       0.0|
|  0.0|(13424,[0,1,2,41,...|[-1078.2664134037...|[1.0,1.5430400222...|       0.0|
|  0.0|(13424,[0,1,4,50,...|[-824.84399055472...|[1.0,8.6563091943...|       0.0|
|  0.0|(13424,[0,1,5,15,...|[-998.47677543575...|[1.0,1.1891868077...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-1154.7968412897...|[1.0,1.0554706203...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-655.83418276928...|[1.0,1.8826740252...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-536.29441760097...|[1.0,3.7326683463...|       0.0|
|  0.0|(13424,[0,1,11,32...|[-868.52533465366...|[1.0,1.6290867695...|       0.0|
|  0.0|(13424,[0,1,12,33...|[-447.73049338017...|[1.0,1.8571715621...|       0.0|
|  0.0|(13424,[0

In [35]:
test_results.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  211|
|  0.0|       1.0|  154|
|  1.0|       0.0|    3|
|  0.0|       0.0| 1310|
+-----+----------+-----+



In [36]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [37]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.9160929147682272
